In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [121]:
def choke_var(bp_won, bp_tot, non_won, non_tot, serve):
    bp_pct = (bp_won + (60 if serve else 40)) / (bp_tot + 100)
    non_pct = (non_won + (64 if serve else 36)) / (non_tot + 100)
    variance = bp_pct * (1 - bp_pct) / (bp_tot * 2 + 200) +\
        non_pct * (1 - non_pct) / (non_tot * 2 + 200)
    return variance

In [7]:
data = pd.read_csv(f"../data/all.csv")

In [122]:
data['var_receive'] = choke_var(data['receive_break_won'], data['receive_break_faced'], data['receive_nobreak_won'], data['receive_nobreak_faced'], True)
data['var_serve'] = choke_var(data['serve_break_won'], data['serve_break_faced'], data['serve_nobreak_won'], data['serve_nobreak_faced'], False)
data['var_combined'] = (data['var_receive'] ** 2 + data['var_serve'] ** 2) ** 0.5 / 2
data

,Unnamed: 0,receive_nobreak_won,receive_nobreak_faced,serve_nobreak_won,serve_nobreak_faced,receive_break_won,receive_break_faced,serve_break_won,serve_break_faced,serve_nobreak_pct,...,receive_break_pct,serve_choke_pct,receive_choke_pct,choke_pct,sd_receive,sd_serve,sd_combined,var_receive,var_serve,var_combined
0,A Ward,37,153,83,164,1,2,19,31,0.556818,...,0.401961,-0.066235,-0.073423,-0.069829,0.040649,0.037599,0.027686,0.001652,0.001414,0.001087
1,A Golubev,146,434,225,456,15,32,53,90,0.519784,...,0.416667,-0.094953,-0.035843,-0.065398,0.033953,0.029691,0.022552,0.001153,0.000882,0.000726
2,R Schuettler,36,134,74,153,1,2,20,33,0.545455,...,0.401961,-0.076049,-0.054268,-0.065159,0.041247,0.037637,0.027919,0.001701,0.001417,0.001107
3,O Rochus,72,323,198,357,3,10,26,47,0.573304,...,0.390909,-0.031730,-0.095590,-0.063660,0.037016,0.033389,0.024925,0.001370,0.001115,0.000883
4,T De Bakker,116,426,280,480,7,19,46,70,0.593103,...,0.394958,-0.050426,-0.065985,-0.058205,0.035323,0.030805,0.023434,0.001248,0.000949,0.000784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,N Milojevic,110,260,119,184,15,42,16,28,0.644366,...,0.387324,0.030616,0.058232,0.044424,0.034990,0.037387,0.025603,0.001224,0.001398,0.000929
444,I Ivashka,488,1302,818,1262,47,130,62,113,0.647577,...,0.378261,0.054807,0.035491,0.045149,0.025020,0.025916,0.018011,0.000626,0.000672,0.000459
445,E Ruusuvuori,397,974,569,941,51,138,61,117,0.608069,...,0.382353,0.030465,0.060813,0.045639,0.025237,0.026272,0.018215,0.000637,0.000690,0.000470
446,M Fish,986,2519,1622,2386,98,265,98,157,0.678198,...,0.378082,0.043412,0.052143,0.047778,0.019549,0.022987,0.015088,0.000382,0.000528,0.000326


comparing the variance in choke_pct the tennis players with the variance expected if everything is just random

we see that there is *slightly* more variance than expected on the serve, but it is virtually identical (slightly less, even) on the return, and in fact the combined choking percentage has *less* variance than expected.

i don't know if this is the mathematically correct way to estimate expected variance of a bunch of random variables, but surely it's close.

In [123]:
{
  'serve': np.var(data['serve_choke_pct']),
  'receive': np.var(data['receive_choke_pct']),
  'combined': np.var(data['choke_pct']),
  'serve_expected': np.mean(data['var_serve']),
  'receive_expected': np.mean(data['var_receive']),
  'combined_expected': np.mean(data['var_combined'])
}

{'serve': 0.0009571588335483506,
 'receive': 0.0009186938408721496,
 'combined': 0.00047267649117717695,
 'serve_expected': 0.0008532994905209682,
 'receive_expected': 0.0009197909081987007,
 'combined_expected': 0.0006279525407645809}

graph idea: generate several random laplace-smoothed choke pcts for each player from a binomial distribution based on their # of points faced/won (basically bootstrap), and plot a histogram of that. then overlap it with the actual histogram of choke pcts.

also maybe see what it looks like without laplace smoothing?